In [1]:
pip install duckduckgo_search langchain_groq sentence_transformers faiss-cpu pydantic python-dotenv pypdf bs4 langchain langchain_core langchain-community langchain_huggingface langchain_google_genai langchain-pinecone langgraph langchain-openai langchain_groq langchain_google_genai google-generativeai  pymupdf wikipedia arxiv defusedxml  youtube_search langchain-tavily langgraph chromadb



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#### import
import os
import pprint
from dotenv import load_dotenv
from langgraph.graph import Graph
from IPython.display import Image, display
load_dotenv()


True

In [3]:
### Configure the Model
from langchain_groq import ChatGroq
model = ChatGroq(model="deepseek-r1-distill-llama-70b")
query = "Hi"
response = model.invoke(query)
pprint.pprint(response.content)



'<think>\n\n</think>\n\nHello! How can I assist you today?'


In [4]:
### Configure the Embedding
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
embed_query = embeddings.embed_query(query)
len(embed_query)


/Users/anvvsharma/anvv/ws/agenticai/vagenticai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384

In [5]:
# Lets take a Data embedd it and store in Vector Database
##### Imports
import requests
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import operator
from langchain_core.tools import tool
from langchain.tools import tool
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,END,MessagesState,START
from langgraph.prebuilt import ToolNode

In [72]:
# Travel Agent Tool
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import re


def travel_agent(state: dict) -> dict:
    """Travel Agent Tool to assist with trip planning."""
    user_messages = state["messages"]
    last_user_message = user_messages[-1].content if user_messages else ""
    response = f"Travel Agent: I can help plan your trip. You asked: {last_user_message}"
    return {
        "messages": user_messages + [AIMessage(content=response)]
    }
    
class WeatherAgent:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5/weather"
    @tool
    def get_weather_city(self, city: str) -> str:
        """Provides weather information for a city."""
        params = {
        "q" : city,
        "appid" : self.api_key,
        "units" : "metric"
        }
        response = requests.get(self.base_url, params=params)    
        if response.status_code == 200:
                
            data = response.json()
            weather_desc = data["weather"][0]["description"]
            temp = data["main"]["temp"]
            humidity = data["main"]["humidity"]
            wind_speed = data["wind"]["speed"]
            return {
                "city": city,
                "description": weather_desc,
                "temperature_celsius": temp,
                "humidity": humidity,
                "wind_speed_m_s": wind_speed
            }
        else:
            return {"error": f"Coold not retrieve weather data for {city}"}
        #weather = Weather(api_key=weather_api_key)
        return f"Weather Agent: The Weather is {city} is sunny and 30 degrees Celsius."
                

class AttrationAgent:
    search = DuckDuckGoSearchRun()
    @staticmethod
    @tool
    def get_attraction_details(city: str) -> str:
        """Provides top attractions for a city."""
        city = city
        query = f"top 5 tourist attractions in {city}"
        results = AttrationAgent.search(query)
        # Use regex to extract lines that look like "1. Place Name" or similar
        place_names = re.findall(r'\d+\.\s*([A-Za-z0-9\s\'\-&]+)', results)
        # Remove duplicates and strip whitespace
        unique_places = []
        for name in place_names:
            clean_name = name.strip()
            if clean_name and clean_name not in unique_places:
                unique_places.append(clean_name)
        # Take top 5
        top_places = unique_places[:5]
        if top_places:
            return f"Attraction Agent: The top 5 attractions in {city} are: {', '.join(top_places)}."
        else:
            return f"Attraction Agent: Sorry, couldn't extract attractions for {city}."
        
    
class CurrencyAgent:
    @tool
    def get_currency_exchange_rate(currency: str) -> str:
        """Provides currency exchange rates."""
        return f"Currency Agent: The current exchange rate for {currency} is 1 {currency} = 1.2 USD."
    
search = DuckDuckGoSearchRun()
# search.invoke("What is the latest update on iPhone 17 relase ?")

# Register all agents/tools
mytools = [
    travel_agent, 
    WeatherAgent.get_weather_city, 
    AttrationAgent.get_attraction_details, 
    CurrencyAgent.get_currency_exchange_rate, 
    
]

llm_Trip_Agent = model.bind_tools(mytools)

SYSTEM_PROMPT = (
    "You are a helpful Travel Planner Agent. "
    "You can use the following agents: travel_agent, weather_agent, attraction_agent, currency_converter, search. "
    "Route the user's query to the correct agent(s) and combine their responses as needed."
)

# Supervisor orchestrates the workflow
class supervisor:
    """Supervisor to manage the workflow of the agents."""
    
    def __init__(self, state: MessagesState):
        self.state = state

    def __call__(self) -> MessagesState:
        return self.invoke(self.state)

    def invoke(self, state: MessagesState) -> MessagesState:
        """Invoke the supervisor to manage the workflow."""
        return supervisor(state)
from langgraph.graph import StateGraph, END, MessagesState, START
from langgraph.prebuilt import ToolNode
# Define the supervisor function


def supervisor(state: MessagesState):
    """Supervisor to manage the workflow of the agents."""
    
    # Extract user messages from the state
    USER_QUERY = state["messages"]
    
    # Prepare the input query for the LLM
    INPUT_QUERY = [SYSTEM_PROMPT] + USER_QUERY
    
    # Invoke the LLM with the input query
    response = llm_Trip_Agent.invoke(INPUT_QUERY)
    
    # Return the response as a new state
    return {
        "messages": [response]
    }


In [73]:

# Build the workflow graph
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from IPython.display import Image, display

TravelAgent_workflow_builder = StateGraph(MessagesState)
TravelAgent_workflow_builder.add_node("supervisor", supervisor)
TravelAgent_workflow_builder.add_node("TravelAgent", travel_agent)
TravelAgent_workflow_builder.add_node("tools", ToolNode(mytools))
TravelAgent_workflow_builder.add_edge(START, "supervisor")
TravelAgent_workflow_builder.add_edge("supervisor", "TravelAgent")
TravelAgent_workflow_builder.add_conditional_edges("TravelAgent", tools_condition,)
TravelAgent_workflow_builder.add_edge("tools","TravelAgent")
TravelAgent_workflow_builder.add_edge("TravelAgent","supervisor")
## React and Action
react_graph = TravelAgent_workflow_builder.compile()
#react_graph

In [75]:
#message = [HumanMessage(content="What are the top attractions and weather in Hyderabad?")]

message = [HumanMessage(content="What are the top attractions in Hyderabad?")]
response = react_graph.invoke({"messages": message})

for m in response["messages"]:
    m.pretty_print()

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [76]:
message = [HumanMessage(content="TravelAgent, what is the waetehr in Hyderabad and Attractions in Hyderabad")]
response = react_graph.invoke({"messages": message})
##### Final Answer 
for m in response["messages"]:
    m.pretty_print()

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT